# Inicializace

In [77]:
from collections import deque               # Ukladani stavu
from __future__ import division             # Deleni realnych cisel (kvuli verzi Pythonu 2.6)
import sys                                  # Pro navazani knihovny Open AI Gym
import numpy as np                          # Knihovna pro operace s maticemi
sys.path.append('/home/xbucha02/libraries') # Adresa knihovny Open AI Gym
import gym                                  # Knihovna Open AI Gym
from gym import wrappers                    # Pomucka na nahravani
env = gym.make('CartPole-v0')            # Konkretni hra z Open AI Gym
env = wrappers.Monitor(env, '/home/lachubcz/tmp/cartpole-experiment-1', force=True) #Nahravani zaznamu
gpuMemoryUsage=1                            # Vyuziti pameti GPU
import tensorflow as tf                     # Knihovna TensorFlow pro sdileni GPU
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = gpuMemoryUsage
set_session(tf.Session(config=config))
import random

# Parametry
observetime = 500                  # Pocet framu kazde hry
startEpsilon = 1                   # Pravdepodobnost konani nahodneho tahu na zacatku
endEpsilon = 0.1                   # Pravdepodobnost konani nahodneho tahu na konci
currentEpsilon = startEpsilon      # Soucasna pravdepodobnost konani nahodneho tahu
epsilonDiminution = 0.995          # Hodnota snizovani epsilonu
gamma = 0.95                       # Snizujici faktor
minibatchSize = 32                 # Velikost minibatche
episodes = 5000                    # Pocet epizod
#numberOfGames = 20                # Pocet her na konci
actionCount = env.action_space.n   # Pocet akci v danem prostredi
stateSize = env.observation_space.shape[0]
learningRate = 0.001             # Learning rate
memorySize = 2000                  # Replay pamet

# Model NN

In [78]:
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics

def getDQN(actionCount,stateSize):
    net = Sequential()
    #net.add(Dense(20, activation="relu", kernel_initializer="uniform", input_shape=(2, 2)))
    #net.add(Flatten())
    net.add(Dense(24, activation="relu", input_dim=stateSize))
    #net.add(Flatten())
    net.add(Dense(24, activation="relu"))
    #net.add(Dropout(rate=0.5))
    #net.add(Dense(8, activation="relu", kernel_initializer="uniform"))
    
    net.add(Dense(actionCount, activation="linear"))
    
    net.summary()
    net.compile(loss=losses.mean_squared_error, optimizer=optimizers.Adam(lr=learningRate), metrics=['accuracy'])
    
    return net

In [79]:
net = getDQN(actionCount, stateSize)
observation = env.reset()                    # Reset prostredi
memory = deque(maxlen=memorySize)

for eps in range (episodes):  
    #stateF = state[0][1] #np.reshape(state, [1, stateSize])
    #D = deque() #vyprazdneni D
    state = env.reset()
    state = np.reshape(state, [1, stateSize])
    env.render()
    #obs = np.expand_dims(observation, axis=0) 
    #state = np.stack((obs, obs), axis=1)

    i = 0
    for t in range(observetime):
        #if i == 0:
            
        #else:
        #    if i==4:
        #        i = 0
        #    else:
        #        i = i + 1
        if np.random.rand() <= currentEpsilon:
            action = np.random.randint(0, actionCount, size=1)[0]
        else:
            #print("shirt")
            Q = net.predict(state)
            #print("shirt")
            action = np.argmax(Q)
            #print("{}".format(action))
        #print ("state: {}" .format(state))
        #print("{}".format(action))
        
        #state = np.reshape(state[0][1], [1, stateSize])
        #print ("state: {}" .format(state))
        observation_new, reward, done, info = env.step(int(action))
        
        #print()
        #stateNewF = observation_new
        #print("heje{}".format(stateNewF))
        #obs_new = np.expand_dims(observation_new, axis=0)        
        #print ("obs_new: {}" .format(obs_new))
        #state_new = np.append(np.expand_dims(obs_new, axis=0), state[:, :1, :], axis=1)
        #state = state[0][0]#next_state = np.reshape(next_state, [1, stateSize])
        next_state = np.reshape(observation_new, [1, stateSize])
        #next_state = [[next_state]]
        memory.append((state, action, reward, next_state, done))
        #print(state, action, reward, stateNewF, done)
        #print ("state: {} state_new: {}" .format(state, next_state))
        #print("state:{}" .format(state))
        state = next_state         # Update state
        #print ("state: {}" .format(state))
        #print ("__________________________________________")
        if done:
            print("Episode: {}/{}, score: {}".format(eps, episodes, t))
            break
    if eps > 0:
        if len(memory) < minibatchSize:
            continue
        minibatch = random.sample(memory, minibatchSize) #z D vybere pocet mb_size samplu

        #inputs_shape = (minibatchSize,) + state.shape[1:]
        #print("inputs_shape: {}".format(inputs_shape))
        #inputs = np.zeros(inputs_shape)
        #print("inputs: {}".format(inputs))
        #targets = np.zeros((minibatchSize, env.action_space.n)) #vytvori pole ([], []), 1. argument - radky, 2. argument - sloupce
        #print("targets: {}".format(targets))

        for i in range(0, minibatchSize):
            #print("{}" .format(minibatch[0][0]))
            stateMb = minibatch[i][0]
            actionMb = minibatch[i][1]
            rewardMb = minibatch[i][2]
            state_newMb = minibatch[i][3]
            doneMb = minibatch[i][4]
            #print("{}".format(rewardMb))
            #print("{}".format(doneMb))
            #target = reward

            # Build Bellman equation for the Q function
            #inputs[i:i+1] = np.expand_dims(state, axis=0)

            #print("inputs: {}".format(inputs))
            #targets[i] = net.predict(state)
            #print("targets: {}".format(targets))
            #print("{}".format(state_newMb))
            #print("{}" .format(state_newMb))
            Q_sa = net.predict(state_newMb)#[0]
            #print("{}, {}".format(state_newMb, Q_sa))
            if doneMb:
                targetMb = rewardMb
            else:
                targetMb = rewardMb + gamma * np.max(Q_sa)
                
            #print("{}".format(targetMb))
            #print("{}" .format(stateMb))
            target_f = net.predict(stateMb)
            #print("1{}".format(target_f))
            target_f[0][actionMb] = targetMb
            #print("1{}".format(stateMb))
            #print("2{}".format(target_f))
            #print("inputs: {}, targets:{}" .format(inputs, targets))

            net.fit(stateMb, target_f, epochs=1, verbose=0)

        #net.train_on_batch(inputs, targets)

        if currentEpsilon > endEpsilon:
            currentEpsilon = currentEpsilon * epsilonDiminution
        print("{}".format(currentEpsilon))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_76 (Dense)             (None, 24)                120       
_________________________________________________________________
dense_77 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_78 (Dense)             (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________
Episode: 0/5000, score: 11
Episode: 1/5000, score: 10
Episode: 2/5000, score: 17
0.995
Episode: 3/5000, score: 15
0.990025
Episode: 4/5000, score: 12
0.985074875
Episode: 5/5000, score: 15
0.980149500625
Episode: 6/5000, score: 10
0.975248753122
Episode: 7/5000, score: 47
0.970372509356
Episode: 8/5000, score: 23
0.965520646809
Episode: 9/5000, score: 47
0.960693043575
Episode: 10/5000, score: 52
0.955889

0.420138972524
Episode: 175/5000, score: 21
0.418038277662
Episode: 176/5000, score: 29
0.415948086273
Episode: 177/5000, score: 75
0.413868345842
Episode: 178/5000, score: 23
0.411799004113
Episode: 179/5000, score: 24
0.409740009092
Episode: 180/5000, score: 73
0.407691309047
Episode: 181/5000, score: 99
0.405652852502
Episode: 182/5000, score: 55
0.403624588239
Episode: 183/5000, score: 43
0.401606465298
Episode: 184/5000, score: 31
0.399598432971
Episode: 185/5000, score: 37
0.397600440806
Episode: 186/5000, score: 62
0.395612438602
Episode: 187/5000, score: 148
0.393634376409
Episode: 188/5000, score: 71
0.391666204527
Episode: 189/5000, score: 29
0.389707873505
Episode: 190/5000, score: 35
0.387759334137
Episode: 191/5000, score: 28
0.385820537467
Episode: 192/5000, score: 17
0.383891434779
Episode: 193/5000, score: 26
0.381971977605
Episode: 194/5000, score: 119
0.380062117717
Episode: 195/5000, score: 48
0.378161807129
Episode: 196/5000, score: 35
0.376270998093
Episode: 197/50

Episode: 360/5000, score: 126
0.165381142455
Episode: 361/5000, score: 146
0.164554236743
Episode: 362/5000, score: 156
0.163731465559
Episode: 363/5000, score: 126
0.162912808231
Episode: 364/5000, score: 162
0.16209824419
Episode: 365/5000, score: 80
0.161287752969
Episode: 366/5000, score: 117
0.160481314204
Episode: 367/5000, score: 197
0.159678907633
Episode: 368/5000, score: 199
0.158880513095
Episode: 369/5000, score: 164
0.158086110529
Episode: 370/5000, score: 199
0.157295679977
Episode: 371/5000, score: 119
0.156509201577
Episode: 372/5000, score: 127
0.155726655569
Episode: 373/5000, score: 196
0.154948022291
Episode: 374/5000, score: 138
0.15417328218
Episode: 375/5000, score: 123
0.153402415769
Episode: 376/5000, score: 161
0.15263540369
Episode: 377/5000, score: 191
0.151872226672
Episode: 378/5000, score: 155
0.151112865538
Episode: 379/5000, score: 129
0.150357301211
Episode: 380/5000, score: 167
0.149605514704
Episode: 381/5000, score: 199
0.148857487131
Episode: 382/5

0.099682091818
Episode: 544/5000, score: 99
0.099682091818
Episode: 545/5000, score: 118
0.099682091818
Episode: 546/5000, score: 199
0.099682091818
Episode: 547/5000, score: 199
0.099682091818
Episode: 548/5000, score: 154
0.099682091818
Episode: 549/5000, score: 142
0.099682091818
Episode: 550/5000, score: 18
0.099682091818
Episode: 551/5000, score: 199
0.099682091818
Episode: 552/5000, score: 140
0.099682091818
Episode: 553/5000, score: 96
0.099682091818
Episode: 554/5000, score: 199
0.099682091818
Episode: 555/5000, score: 199
0.099682091818
Episode: 556/5000, score: 116
0.099682091818
Episode: 557/5000, score: 85
0.099682091818
Episode: 558/5000, score: 127
0.099682091818
Episode: 559/5000, score: 199
0.099682091818
Episode: 560/5000, score: 199
0.099682091818
Episode: 561/5000, score: 78
0.099682091818
Episode: 562/5000, score: 54
0.099682091818
Episode: 563/5000, score: 143
0.099682091818
Episode: 564/5000, score: 161
0.099682091818
Episode: 565/5000, score: 177
0.099682091818
E

0.099682091818
Episode: 727/5000, score: 109
0.099682091818
Episode: 728/5000, score: 88
0.099682091818
Episode: 729/5000, score: 90
0.099682091818
Episode: 730/5000, score: 81
0.099682091818
Episode: 731/5000, score: 89
0.099682091818
Episode: 732/5000, score: 71
0.099682091818
Episode: 733/5000, score: 71
0.099682091818
Episode: 734/5000, score: 115
0.099682091818
Episode: 735/5000, score: 113
0.099682091818
Episode: 736/5000, score: 16
0.099682091818
Episode: 737/5000, score: 158
0.099682091818
Episode: 738/5000, score: 199
0.099682091818
Episode: 739/5000, score: 161
0.099682091818
Episode: 740/5000, score: 83
0.099682091818
Episode: 741/5000, score: 199
0.099682091818
Episode: 742/5000, score: 29
0.099682091818
Episode: 743/5000, score: 137
0.099682091818
Episode: 744/5000, score: 114
0.099682091818
Episode: 745/5000, score: 13
0.099682091818
Episode: 746/5000, score: 125
0.099682091818
Episode: 747/5000, score: 128
0.099682091818
Episode: 748/5000, score: 135
0.099682091818
Episo

Episode: 910/5000, score: 155
0.099682091818
Episode: 911/5000, score: 199
0.099682091818
Episode: 912/5000, score: 185
0.099682091818
Episode: 913/5000, score: 199
0.099682091818
Episode: 914/5000, score: 199
0.099682091818
Episode: 915/5000, score: 178
0.099682091818
Episode: 916/5000, score: 85
0.099682091818
Episode: 917/5000, score: 139
0.099682091818
Episode: 918/5000, score: 160
0.099682091818
Episode: 919/5000, score: 176
0.099682091818
Episode: 920/5000, score: 140
0.099682091818
Episode: 921/5000, score: 145
0.099682091818
Episode: 922/5000, score: 199
0.099682091818
Episode: 923/5000, score: 192
0.099682091818
Episode: 924/5000, score: 130
0.099682091818
Episode: 925/5000, score: 188
0.099682091818
Episode: 926/5000, score: 102
0.099682091818
Episode: 927/5000, score: 199
0.099682091818
Episode: 928/5000, score: 177
0.099682091818
Episode: 929/5000, score: 180
0.099682091818
Episode: 930/5000, score: 162
0.099682091818
Episode: 931/5000, score: 103
0.099682091818
Episode: 93

Episode: 1091/5000, score: 199
0.099682091818
Episode: 1092/5000, score: 108
0.099682091818
Episode: 1093/5000, score: 137
0.099682091818
Episode: 1094/5000, score: 199
0.099682091818
Episode: 1095/5000, score: 199
0.099682091818
Episode: 1096/5000, score: 199
0.099682091818
Episode: 1097/5000, score: 174
0.099682091818
Episode: 1098/5000, score: 149
0.099682091818
Episode: 1099/5000, score: 122
0.099682091818
Episode: 1100/5000, score: 181
0.099682091818
Episode: 1101/5000, score: 138
0.099682091818
Episode: 1102/5000, score: 131
0.099682091818
Episode: 1103/5000, score: 123
0.099682091818
Episode: 1104/5000, score: 118
0.099682091818
Episode: 1105/5000, score: 199
0.099682091818
Episode: 1106/5000, score: 144
0.099682091818
Episode: 1107/5000, score: 134
0.099682091818
Episode: 1108/5000, score: 183
0.099682091818
Episode: 1109/5000, score: 176
0.099682091818
Episode: 1110/5000, score: 191
0.099682091818
Episode: 1111/5000, score: 185
0.099682091818
Episode: 1112/5000, score: 156
0.0

Episode: 1270/5000, score: 199
0.099682091818
Episode: 1271/5000, score: 196
0.099682091818
Episode: 1272/5000, score: 199
0.099682091818
Episode: 1273/5000, score: 199
0.099682091818
Episode: 1274/5000, score: 199
0.099682091818
Episode: 1275/5000, score: 198
0.099682091818
Episode: 1276/5000, score: 199
0.099682091818
Episode: 1277/5000, score: 199
0.099682091818
Episode: 1278/5000, score: 199
0.099682091818
Episode: 1279/5000, score: 199
0.099682091818
Episode: 1280/5000, score: 115
0.099682091818
Episode: 1281/5000, score: 186
0.099682091818
Episode: 1282/5000, score: 13
0.099682091818
Episode: 1283/5000, score: 184
0.099682091818
Episode: 1284/5000, score: 143
0.099682091818
Episode: 1285/5000, score: 163
0.099682091818
Episode: 1286/5000, score: 150
0.099682091818
Episode: 1287/5000, score: 146
0.099682091818
Episode: 1288/5000, score: 155
0.099682091818
Episode: 1289/5000, score: 144
0.099682091818
Episode: 1290/5000, score: 165
0.099682091818
Episode: 1291/5000, score: 121
0.09

Episode: 1449/5000, score: 190
0.099682091818
Episode: 1450/5000, score: 168
0.099682091818
Episode: 1451/5000, score: 173
0.099682091818
Episode: 1452/5000, score: 194
0.099682091818
Episode: 1453/5000, score: 199
0.099682091818
Episode: 1454/5000, score: 175
0.099682091818
Episode: 1455/5000, score: 174
0.099682091818
Episode: 1456/5000, score: 178
0.099682091818
Episode: 1457/5000, score: 187
0.099682091818
Episode: 1458/5000, score: 199
0.099682091818
Episode: 1459/5000, score: 199
0.099682091818
Episode: 1460/5000, score: 181
0.099682091818
Episode: 1461/5000, score: 167
0.099682091818
Episode: 1462/5000, score: 196
0.099682091818
Episode: 1463/5000, score: 156
0.099682091818
Episode: 1464/5000, score: 154
0.099682091818
Episode: 1465/5000, score: 150
0.099682091818
Episode: 1466/5000, score: 44
0.099682091818
Episode: 1467/5000, score: 143
0.099682091818
Episode: 1468/5000, score: 146
0.099682091818
Episode: 1469/5000, score: 187
0.099682091818
Episode: 1470/5000, score: 154
0.09

KeyboardInterrupt: 